In [1]:
import praw
import re
import json
import numpy as np
import pandas as pd
from copy import deepcopy
import pickle
#Account Age
import time
from datetime import datetime

from sqlalchemy import create_engine

In [2]:
reddit = praw.Reddit(client_id = "",
                    client_secret = "",
                    user_agent="",
                    username = "",
                    password = "")

In [3]:
def scrape_submissions_1000():
    sub_list = []
    subreddit = reddit.subreddit('hardwareswap')
    for submission in subreddit.new(limit=None):
        author = str(submission.author)
        dates = time.strftime('%m/%d/%Y', time.gmtime(submission.created_utc))
        permalink = submission.permalink
        text = submission.selftext
        title = submission.title
        pk = submission.id

        sub_list.append([pk, dates, title, text, author, permalink])
    df = pd.DataFrame(sub_list, columns=['id', 'date', 'title', 'text', 'author', 'link'])
        
    return df

In [ ]:
df = scrape_submissions_1000()

In [ ]:
def new_submissions(df):    
    cur = cnx.cursor()
    cur.execute("""SELECT * FROM Submissions.submissions;""")
    df_sql = pd.DataFrame(cur.fetchall())
    df_sql.columns = [x[0] for x in cur.description]
    df_sql.to_csv('/Users/eric/FlatIron/section01/Projects/reddit_scraper/db_backup/submission_sql_backup.csv')
    new_submissions = df[~df['id'].isin(df_sql['id'])]
    new_sub_list = new_submissions.values.tolist()
    return new_sub_list, new_submissions

In [ ]:
new_sub_list, new_submissions = new_submissions(df)

## SQL

In [ ]:
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.passwd,
        database = config.db_name
)

print(cnx)
cursor = cnx.cursor()

In [ ]:
engine = create_engine("mysql+mysqlconnector://{user}:{pw}@{host}/{db}"
                       .format(user=config.user,
                               pw=config.passwd,
                               host=config.host,
                               db=config.db_name))

In [ ]:
# Insert whole DataFrame into MySQL
new_submissions.to_sql('submissions', con = engine, if_exists = 'append')